# 1.understanding the first populationg orca.table "nodes" step

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os; 
os.chdir('..')
os.chdir('..')
import models
import datasources
import variables
import orca
import pandas as pd
import numpy as np

## [1] Here pay attention that price_vars orca.step shall be running after calling rsh_simulate and rrh_simulate
* "neighborhood_vars":accessibility variables
* "prices_vars": node-level variables for feasibility
normally should be run after hedonic regression

## [2] What are rsh_simulate and rrh_simulate?

```python
@orca.step('rsh_simulate')
def rsh_simulate(residential_units, unit_aggregations):
    return utils.hedonic_simulate("rsh.yaml", residential_units, 
    								unit_aggregations, "unit_residential_price")


@orca.step('rrh_estimate')
def rh_cl_estimate(craigslist, aggregations):
    return utils.hedonic_estimate("rrh.yaml", craigslist, aggregations)

```
## [3] Also in the simulations.py file

```python
try:
  orca.run([ 
    "neighborhood_vars",         # accessibility variables
    
    "rsh_simulate",              # residential sales hedonic
    "rrh_simulate",              # residential rental hedonic
    "nrh_simulate",              # non-residential rent hedonic

    #"households_relocation",
    "households_relocation_filtered",
    "households_transition",
    
    "hlcm_owner_simulate",       # location choice model for owners
    "hlcm_renter_simulate",      # location choice model for renters
    #"hlcm_simulate",
    #"hlcm_li_simulate",

    "jobs_relocation",
    "jobs_transition",
    "elcm_simulate",

    "price_vars",				# node-level variables for feasibility
    "price_to_rent_precompute",	# translate rental/ownership income into consistent terms

    "feasibility",				# calculate feasibility of all new building types
    
    "scheduled_development_events", # scheduled buildings additions
    "residential_developer",
    "non_residential_developer",
     
    "diagnostic_output",
    "travel_model_output"
], iter_vars=range(in_year, out_year))

```


In [3]:
orca.run([
    "neighborhood_vars",
    "rsh_simulate",
    "rrh_simulate",
    "price_vars",
], iter_vars=[2010])

Running iteration 1 with iteration value 2010
Running step 'neighborhood_vars'
Computing accessibility variables
Computing sum_residential_units
Computing sum_nonresidential_units
Computing ave_sqft_per_unit
Computing ave_lot_size_per_unit
Computing population
Computing poor
Computing blacks
Removed 3495 rows because they contain missing values
Computing whites
Removed 28069 rows because they contain missing values
Computing nonwhites
Removed 49824 rows because they contain missing values
Computing hispanics
Removed 5989 rows because they contain missing values
Computing asians
Removed 7970 rows because they contain missing values
Computing renters
Computing singles
Removed 49332 rows because they contain missing values
Computing two_persons
Removed 13712 rows because they contain missing values
Computing three_plus
Removed 14849 rows because they contain missing values
Computing sfdu
Computing ave_hhsize
Computing jobs
Computing jobs3k
Computing ave_income
       sum_residential_units

In [4]:
hh = orca.get_table('households').to_frame()
hh[1:5]

,taz,serialno,puma5,income,persons,hht,unittype,noc,bldgsz,tenure,...,income_quartile,positive_income,node_id,black,asian,hisp,white,hhs3p,hhs1,hh_monthly_rent
household_id,,,,,,,,,,,,,,,,,,,,,
2687058,1092,8097339,2103,0,1,0,2,0,0,0,...,1,1,NaN,0,0,0,0,False,True,0
2687063,1092,3530666,2103,0,1,0,2,0,0,0,...,1,1,NaN,0,0,0,0,False,True,0
2214782,930,5691854,2402,159700,3,5,0,0,2,1,...,4,159700,NaN,0,0,0,1,True,False,0
2687065,1092,8097339,2103,0,1,0,2,0,0,0,...,1,1,NaN,0,0,0,0,False,True,0
